In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def check_status(link):
    try:
        request = requests.get(link)
        request.raise_for_status()  # returns an HTTPError if the response is not OK
        print("All good! Response code is", request.status_code)
    except requests.exceptions.HTTPError as err:
        if request.status_code == 404:
            print("404: Oops, sorry we can't find that page!")
        else:
            print("The error code is", err.args[0]) # look up the 1st argument from HTTPError
        return None

In [ ]:
teams_url = "https://api-football-v1.p.rapidapi.com/v3/teams"
leagues_url = "https://api-football-v1.p.rapidapi.com/v3/leagues"
squad_url = "https://api-football-v1.p.rapidapi.com/v3/players/squads"
standings_url = "https://api-football-v1.p.rapidapi.com/v3/standings"
players_url= "https://api-football-v1.p.rapidapi.com/v3/players"

In [ ]:
headers = {"X-RapidAPI-Key": "1172bcc0b2msh9e4885009157365p16ecbejsn0c707804a0bb",
           "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"}

In [ ]:
leagues_response=requests.get(leagues_url, headers=headers)
leagues_data=leagues_response.json()

In [ ]:
leagues_data

In [ ]:
leagues_data.keys()

In [ ]:
leagues_data["response"]

In [ ]:
leagues_data["response"][0]

In [ ]:
leagues_data["response"][0]["league"]

In [ ]:
leagues_data["response"][0]["league"]["id"]

In [ ]:
leagues_data["response"][0]

In [ ]:
premier_league_id = None
for league in leagues_data["response"]:
    if league["league"]["name"] == "Premier League":
        premier_league_id = league["league"]["id"]
        break

In [ ]:
premier_league_id

In [ ]:
year=input("Which year do you want to get information? ")
parameters = {"league":premier_league_id, "season":year}
teams_response = requests.get(teams_url, headers=headers, params=parameters)
teams_data = teams_response.json()
teams_data

In [ ]:
teams_data["response"]

In [ ]:
teams_data_id = []
teams_data_name = []
for team in teams_data["response"]:
    team_id = team["team"]["id"]
    team_name = team["team"]["name"]
    teams_data_id.append(team_id)
    teams_data_name.append(team_name)

In [ ]:
teams_data_name

In [ ]:
teams_df= pd.DataFrame({"Team ID": teams_data_id, "Team Name": teams_data_name})
teams_df

In [ ]:
team_id = input("Which Team ID do you want to know more about? ") 
querystring={"team":team_id}

headers = {
	"X-RapidAPI-Key": "1172bcc0b2msh9e4885009157365p16ecbejsn0c707804a0bb",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

squad_response = requests.get(squad_url, headers=headers, params=querystring)
squad_data=squad_response.json()

In [ ]:
team_id_squad=squad_data["response"][0]["players"]
team_id_squad

In [ ]:
team_id_squad_df=pd.DataFrame(team_id_squad)
team_id_squad_df

In [ ]:
average_age_team_id_squad_df=team_id_squad_df["age"].mean()
average_age_team_id_squad_df

In [ ]:
your_answer=input("In what year would you like to know who was champion? ")
querystring_1 = {"season":your_answer,"league":premier_league_id}

headers = {
	"X-RapidAPI-Key": "1172bcc0b2msh9e4885009157365p16ecbejsn0c707804a0bb",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

standings_response = requests.get(standings_url, headers=headers, params=querystring_1)
standings_data=standings_response.json()

In [ ]:
league_winner = standings_data["response"][0]["league"]["standings"][0][0]["team"]["name"]
if league_winner:
    print(f"The league winner for {your_answer} is: {league_winner}")
else:
    print(f"No league winner found for {your_answer}")

In [ ]:
#def league_squad_year():
def league_id(your_league):
        leagues_response=requests.get(leagues_url, headers=headers)
        leagues_data=leagues_response.json()
        league_id = None
        for league in leagues_data["response"]:
            if league["league"]["name"] == your_league:
                league_id = league["league"]["id"]
                break
        return league_id       


def team_id(league_id, year):
        parameters = {"league":league_id, "season":year}
        teams_response = requests.get(teams_url, headers=headers, params=parameters)
        teams_data = teams_response.json()
        teams_data_id = []
        teams_data_name = []
        for team in teams_data["response"]:
            team_id = team["team"]["id"]
            team_name = team["team"]["name"]
            teams_data_id.append(team_id)
            teams_data_name.append(team_name)
        teams_df= pd.DataFrame({"Team ID": teams_data_id, "Team Name": teams_data_name})
        return teams_df


def squad_information(teams_data_id):
        all_teams_squad_data = []
        for team_id in teams_data_id:
            parameters={"team":team_id}
            squad_response = requests.get(squad_url, headers=headers, params=parameters)
            squad_data=squad_response.json()
            if squad_data["response"]:
                team_squad_data = squad_data["response"][0]["players"]
                for player in team_squad_data:
                    player['club'] = team_id 
                all_teams_squad_data.extend(team_squad_data)
        all_teams_squad_data_df=pd.DataFrame(all_teams_squad_data)
        return pd.merge(teams_df, all_teams_squad_data_df, on="team_id", how= "outer")


def standings_information(league_id, year):
        parameters = {"season":year,"league":league_id}
        standings_response = requests.get(standings_url, headers=headers, params=querystring_1)
        standings_data=standings_response.json()
        league_winner = standings_data["response"][0]["league"]["standings"][0][0]["team"]["name"]
        if league_winner:
            print(f"The league winner for {your_answer} is: {league_winner}")
        else:
            print(f"No league winner found for {your_answer}")

In [ ]:
your_league = "Premier League"
year = "2023"

In [ ]:
league_id(your_league)

In [ ]:
league_id="39"

In [ ]:
team_id(league_id,year)

In [ ]:
teams_data_id=[33,34,35,36,39,40,42,44,45,47,48,49,50,51,52,55,62,65,66,1359]

In [ ]:
squad_information(teams_data_id)

In [ ]:
leagues_response=requests.get(leagues_url, headers=headers)
leagues_data=leagues_response.json()
your_league=input("Which league do you want to analyze? ")
league_id = None
for league in leagues_data["response"]:
    if league["league"]["name"] == your_league:
        league_id = league["league"]["id"]
        break
league_id

In [ ]:
year=input("Which year do you want to analyze? ")
parameters = {"league":league_id, "season":year}
teams_response = requests.get(teams_url, headers=headers, params=parameters)
teams_data = teams_response.json()
teams_data_id = []
teams_data_name = []
for team in teams_data["response"]:
    team_id = team["team"]["id"]
    team_name = team["team"]["name"]
    teams_data_id.append(team_id)
    teams_data_name.append(team_name)
teams_df= pd.DataFrame({"Team ID": teams_data_id, "Team Name": teams_data_name})
teams_df

In [ ]:
all_teams_squad_data = []
for team_id in teams_data_id:
    parameters={"team":team_id}
    squad_response = requests.get(squad_url, headers=headers, params=parameters)
    squad_data=squad_response.json()
    if squad_data["response"]:
        team_squad_data = squad_data["response"][0]["players"]
        for player in team_squad_data:
            player['Team ID'] = team_id 
        all_teams_squad_data.extend(team_squad_data)
    all_teams_squad_data_df=pd.DataFrame(all_teams_squad_data)
final_pd=pd.merge(teams_df, all_teams_squad_data_df, on="Team ID", how= "outer")
final_pd

In [ ]:
final_pd.info()

In [ ]:
final_pd.dropna(subset=['age'], inplace=True)

In [ ]:
final_pd["age"] = final_pd["age"].astype(int)

In [ ]:
final_pd.info()

In [ ]:
team_average_age=final_pd.groupby("Team Name")["age"].mean()

In [ ]:
sort_team_average_age=team_average_age.sort_values(ascending=True)
sort_team_average_age

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sort_team_average_age.plot(kind='bar')

In [ ]:
parameters = {"season":year,"league":league_id}
standings_response = requests.get(standings_url, headers=headers, params=parameters)
standings_data=standings_response.json()
league_winner = standings_data["response"][0]["league"]["standings"][0][0]["team"]["name"]
league_winner_df=pd.DataFrame({"Year": [year], "Champion": [league_winner]})
league_winner_df

In [ ]:
fixtures_url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"
parameters = {"season":year,"team":team_id}
fixtures_response=requests.get(fixtures_url, headers=headers, params=parameters)
fixtures_data=fixtures_response.json()

In [ ]:
fixture_id=fixtures_data["response"][0]["fixture"]["id"]

In [ ]:
lineups_url = "https://api-football-v1.p.rapidapi.com/v3/fixtures/lineups"

In [ ]:
parameters={"fixture":fixture_id, "team":team_id}
lineups_response=requests.get(lineups_url, headers=headers, params=parameters)
lineups_data=lineups_response.json()

In [ ]:
lineup=lineups_data["response"][0]["startXI"][i]["player"]

In [ ]:
lineups_data["response"][0]["startXI"][0]["player"]["name"]

In [ ]:
players_count={}
parameters = {"season":year,"team":team_id}
fixtures_response=requests.get(fixtures_url, headers=headers, params=parameters)
fixtures_data=fixtures_response.json()
for fixture in fixtures_data["response"]:
    fixture_id=fixture["fixture"]["id"]
    parameters={"fixture":fixture_id, "team":team_id}
    lineups_response=requests.get(lineups_url, headers=headers, params=parameters)
    lineups_data=lineups_response.json()
    if lineups_data["response"]:
        for lineup in lineups_data["response"][0]["startXI"]:
            player_name=lineup["player"]["name"]
            players_count[player_name] = players_count.get(player_name, 0) + 1
lineup_df = pd.DataFrame(players_count, columns=["name", "Appearances"])
lineup_df

In [ ]:
current_year= int(input("Please specify the current year. "))
years_back=int(input("Please specify for how many years do you want to analyze the data? "))
winners_df = []
for year in range(current_year, current_year - years_back - 1, -1):
    parameters = {"season":year,"league":league_id}
    standings_response = requests.get(standings_url, headers=headers, params=parameters)
    standings_data=standings_response.json()
    league_winner = standings_data["response"][0]["league"]["standings"][0][0]["team"]["name"]
    league_winner_df=pd.DataFrame({"Year": [year], "Champion": [league_winner]})
    winners_df.append(league_winner_df)
all_winners_df = pd.concat(winners_df, ignore_index=True)
all_winners_df

In [ ]:
current_year= int(input("Please specify the current year "))
years_back=int(input("Please specify how many winners do you want to know? "))
all_teams = []
for year in range(current_year, current_year - years_back - 1, -1):
    parameters = {"league":league_id, "season":year}
    teams_response = requests.get(teams_url, headers=headers, params=parameters)
    teams_data = teams_response.json()
    teams_data_id = []
    teams_data_name = []
    teams_data_year = []
    for team in teams_data["response"]:
        team_id = team["team"]["id"]
        team_name = team["team"]["name"]
        team_year = year
        teams_data_id.append(team_id)
        teams_data_name.append(team_name)
        teams_data_year.append(team_year)
    teams_df= pd.DataFrame({"Team ID": teams_data_id, "Team Name": teams_data_name, "Year": teams_data_year})
    all_teams.append(teams_df)
all_teams_df = pd.concat(all_teams, ignore_index=True)
all_teams_df

In [ ]:
all_teams_squad_data = []
for year in range(current_year, current_year - years_back - 1, -1):
    for team_id in teams_data_id:
        parameters={"team":team_id, "season":year}
        squad_response = requests.get(squad_url, headers=headers, params=parameters)
        squad_data=squad_response.json()
        if squad_data["response"]:
            team_squad_data = squad_data["response"][0]["players"]
            for player in team_squad_data:
                player['Team ID'] = team_id 
            all_teams_squad_data.extend(team_squad_data)
        all_teams_squad_data_df=pd.DataFrame(all_teams_squad_data)
final_pd=pd.merge(teams_df, all_teams_squad_data_df, on="Team ID", how= "outer")
final_pd

In [ ]:
all_teams_squad_data = []
for year in range(current_year, current_year - years_back - 1, -1):
    for team_id in teams_data_id:
        parameters={"team":team_id, "season":year}
        squad_response = requests.get(squad_url, headers=headers, params=parameters)
        squad_data=squad_response.json()
        team_squad_data = squad_data["response"][0]["players"]
        for player in team_squad_data:
            player['Team ID'] = team_id 
        all_teams_squad_data.extend(team_squad_data)
    all_teams_squad_data_df=pd.DataFrame(all_teams_squad_data)
final_pd=pd.merge(teams_df, all_teams_squad_data_df, on="Team ID", how= "outer")
final_pd

In [ ]:
squad_data["message"]

In [ ]:
teams = ["Arsenal", "Chelsea", "Liverpool", "Manchester City", "Manchester United", "Tottenham"]
team_average_ages = {team: [] for team in teams}
for year in range(2023, 2013, -1):
    for team in teams:
        params = {"season": year, "team": team}
        response = requests.get(squad_url, headers=headers, params=params)
        squad_data = response.json()
        if squad_data["response"]:
            players = squad_data["response"][0]["players"]
            if players:
                ages = [player["age"] for player in players]
                average_age = sum(ages) / len(ages)
                team_average_ages[team].append(average_age)
            else:
                team_average_ages[team].append(None)
        else:
            team_average_ages[team].append(None)
plt.figure(figsize=(10, 6))
for team, average_ages_yearly in team_average_ages_yearly.items():
    plt.plot(range(2022, 2012, -1), average_ages_yearly, label=team)
plt.title("Average Age of Squads for Each Team Over the Past 10 Years")
plt.xlabel("Year")
plt.ylabel("Average Age")
plt.legend()
plt.grid(True)
plt.xticks(range(2022, 2012, -1), rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
teams = ["Arsenal", "Chelsea", "Liverpool", "Manchester City", "Manchester United", "Tottenham"]

# Initialize a dictionary to store average ages for each team in each year
team_average_ages_yearly = {team: [] for team in teams}

# Iterate over the past 10 years
for year in range(2023, 2020, -1):
    for team in teams:
        # Make a request to get the squad for the current team and year
        params = {"season": year, "team": team}
        response = requests.get(squad_url, headers=headers, params=params)
        squad_data = response.json()
        
        # Calculate the average age for the squad
        if squad_data["response"]:
            players = squad_data["response"][0]["players"]
            if players:
                ages = [player["age"] for player in players]
                team_ages_yearly[team].append(ages)
            else:
                team_ages_yearly[team].append(None)
        else:
            team_ages_yearly[team].append(None)

# Create a line plot for each team
plt.figure(figsize=(10, 6))

for team, average_ages_yearly in team_average_ages_yearly.items():
    plt.plot(range(2023, 2020, -1), average_ages_yearly, label=team)

plt.title("Average Age of Squads for Each Team Over the Past 10 Years")
plt.xlabel("Year")
plt.ylabel("Average Age")
plt.legend()
plt.grid(True)
plt.xticks(range(2023, 2020, -1), rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
teams = ["Arsenal", "Chelsea", "Liverpool", "Manchester City", "Manchester United", "Tottenham"]

# Initialize an empty list to store the data
data = []

# Iterate over the past 10 years
for year in range(2023, 2020, -1):
    for team in teams:
        # Make a request to get the squad for the current team and year
        params = {"season": year, "team": team}
        response = requests.get(squad_url, headers=headers, params=params)
        squad_data = response.json()
        
        # Calculate the average age for the squad
        if squad_data["response"]:
            players = squad_data["response"][0]["players"]
            if players:
                ages = [player["age"] for player in players]
                data.append({"Year": year, "Team": team, "Age": average_age})

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

# Group by team and year and calculate the average age for each team by year
team_avg_age_by_year = df.groupby(["Team", "Year"])["Age"].mean().reset_index()

print(team_avg_age_by_year)

In [ ]:
teams = ["Arsenal", "Chelsea", "Liverpool", "Manchester City", "Manchester United", "Tottenham"]

# Initialize a dictionary to store average ages for each team in each year
team_average_ages_yearly = {team: [] for team in teams}

# Iterate over the past 10 years
for year in range(2023, 2019, -1):  # Adjust the range as needed
    for team in teams:
        # Make a request to get the squad for the current team and year
        params = {"season": year, "team": team}
        response = requests.get(squad_url, headers=headers, params=params)
        squad_data = response.json()
        
        # Calculate the average age for the squad
        if squad_data["response"]:
            players = squad_data["response"][0]["players"]
            if players:
                ages = [player["age"] for player in players]
                average_age = sum(ages) / len(ages)
                team_average_ages_yearly[team].append(average_age)
            else:
                team_average_ages_yearly[team].append(None)
        else:
            team_average_ages_yearly[team].append(None)
team_average_ages_yearly_df=pd.DataFrame(team_average_ages_yearly)
team_average_ages_yearly_df

In [ ]:
teams_dict = {}
all_teams_squad_data = []
for team in teams_data["response"]:
    teams_dict[team["team"]["name"]] = team["team"]["id"]

# Define the list of team names
teams = ["Arsenal", "Chelsea", "Liverpool", "Manchester City", "Manchester United", "Tottenham"]

# Initialize a dictionary to store average ages for each team in each year
team_average_ages_yearly = {team: [] for team in teams}

# Define the range of years
start_year = 2023
end_year = 2020

# Iterate over the range of years
for year in range(start_year, end_year, -1):
    for team in teams:
        # Retrieve the team ID from the dictionary
        team_id = teams_dict.get(team)
        if team_id:
            # Make a request to get the squad for the current team and year
            squad_url = "https://api-football-v1.p.rapidapi.com/v3/players/squads"
            parameters={"team":team_id}
            squad_response = requests.get(squad_url, headers=headers)
            squad_data = squad_response.json()
            
            # Check if the response contains data
            if squad_data["response"]:
                team_squad_data = squad_data["response"][0]["players"]
                for player in team_squad_data:
                    player['Team ID'] = team_id 
            all_teams_squad_data.extend(team_squad_data)
    all_teams_squad_data_df=pd.DataFrame(all_teams_squad_data)
final_pd=pd.merge(teams_df, all_teams_squad_data_df, on="Team ID", how= "outer")
final_pd

In [ ]:
plt.plot(df['Year'], df['Average Age'], marker='o')
plt.title('Average Age of Chelsea Squad (2020-2023)')
plt.xlabel('Year')
plt.ylabel('Average Age')
plt.grid(True)
plt.show()


In [ ]:
all_teams_squad_data = []
years=[2021, 2022, 2023]
teams_data_id=[33, 40, 47]
for year in years:
    for team_id in teams_data_id:
        parameters={"team":team_id}
        squad_response = requests.get(squad_url, headers=headers, params=parameters)
        squad_data=squad_response.json()
        if squad_data["response"]:
            team_squad_data = squad_data["response"][0]["players"]
            for player in team_squad_data:
                player['Team ID'] = team_id 
            all_teams_squad_data.extend(team_squad_data)
        all_teams_squad_data_df=pd.DataFrame(all_teams_squad_data)
final_pd=pd.merge(teams_df, all_teams_squad_data_df, on="Team ID", how= "outer")

In [ ]:
all_teams_squad_data = []

# Define the years and teams' IDs
years = [2021, 2022, 2023]
teams_data_id = [33, 40, 47]

# Loop through each year and team ID to fetch squad data
for year in years:
    for team_id in teams_data_id:
        parameters = {"id": team_id, "season": year}
        squad_response = requests.get(squad_url, headers=headers, params=parameters)
        squad_data = squad_response.json()
        if squad_data["response"]:
            team_squad_data = squad_data["response"][0]["players"]
            for player in team_squad_data:
                # Check if 'Team ID' exists in player data, if not, add it
                if 'Team ID' not in player:
                    player['Team ID'] = team_id
            all_teams_squad_data.extend(team_squad_data)

# Create DataFrame from all_teams_squad_data
all_teams_squad_data_df = pd.DataFrame(all_teams_squad_data)

# Assuming teams_df is already defined elsewhere
# Merge teams DataFrame with squad data on 'Team ID'
final_pd = pd.merge(teams_df, all_teams_squad_data_df, on="Team ID", how="outer")

# Display the final DataFrame
print(final_pd)

In [ ]:
final_pd.dropna()

In [ ]:
team_average_age=final_pd.groupby(["Team Name", "Year"])["age"].mean()
team_average_age

In [ ]:
year=input("Which year do you want to analyze? ")
parameters = {"league":league_id, "season":year}
teams_response = requests.get(teams_url, headers=headers, params=parameters)
teams_data = teams_response.json()
teams_data_id = []
teams_data_name = []
for team in teams_data["response"]:
    team_id = team["team"]["id"]
    team_name = team["team"]["name"]
    teams_data_id.append(team_id)
    teams_data_name.append(team_name)
teams_df= pd.DataFrame({"Team ID": teams_data_id, "Team Name": teams_data_name})
teams_df

In [ ]:
all_teams_squad_data = []
for team_id in teams_data_id:
    parameters={"team":team_id, "season":year}
    squad_response = requests.get(squad_url, headers=headers, params=parameters)
    squad_data=squad_response.json()
    if squad_data["response"]:
        team_squad_data = squad_data["response"][0]["players"] 
        all_teams_squad_data.extend(team_squad_data)
    all_teams_squad_data_df=pd.DataFrame(all_teams_squad_data)
final_pd=pd.merge(teams_df, all_teams_squad_data_df, on="Team ID", how= "outer")
team_average_age=final_pd.groupby("Team Name")["age"].mean()
sort_team_average_age=team_average_age.sort_values(ascending=True)
sort_team_average_age
all_teams_squad_data_df

In [ ]:
data=[]
team_name_7 = 'Arsenal'
year_7 = 2022
average_age_7 = 23.72
team_name_8 = 'Chelsea'
year_8 = 2022
average_age_8 = 21.4
team_name_9 = 'Liverpool'
year_9 = 2022
average_age_9 = 23.37
team_name_10 = 'Manchester City'
year_10 = 2022
average_age_10 = 24.96
team_name_11 = 'Manchester United'
year_11 = 2022
average_age_11 = 24.44
team_name_12 = 'Tottenham'
year_12 = 2022
average_age_12 = 24.23
data.append([team_name_7, year_7, average_age_7])
data.append([team_name_8, year_8, average_age_8])
data.append([team_name_9, year_9, average_age_9])
data.append([team_name_10, year_10, average_age_10])
data.append([team_name_11, year_11, average_age_11])
data.append([team_name_12, year_12, average_age_12])
df = pd.DataFrame(data, columns=['Team Name', 'Year', 'Average Age'])
df

In [ ]:
new_row_1=['Arsenal', 2020, 23.72]
new_row_2=['Chelsea', 2020, 21.4]
new_row_3=['Liverpool', 2020, 23.37]
new_row_4=['Manchester City', 2020, 24.96]
new_row_5=['Manchester United', 2020, 24.44]
new_row_6=['Tottenham', 2020, 24.23]
df.loc[len(df)] = new_row_1
df.loc[len(df)] = new_row_2
df.loc[len(df)] = new_row_3
df.loc[len(df)] = new_row_4
df.loc[len(df)] = new_row_5
df.loc[len(df)] = new_row_6
df

In [ ]:
players_squads_url = "https://api-football-v1.p.rapidapi.com/v3/players/seasons"
team_id = "33"
year = "2020"

# Define the parameters
parameters = {"id": team_id, "season": year}

# Make the API request
players_squads_response = requests.get(players_squads_url, headers=headers, params=parameters)
players_squads_data = players_squads_response.json()

# Display the response data
print(players_squads_data)

In [ ]:
team_id = "33"  # Example: Chelsea's team ID
season = "2020"

# Define the parameters
parameters = {"team": team_id, "Season": season}

# Make the API request
squad_response = requests.get(squad_url, headers=headers, params=parameters)
squad_data = squad_response.json()

# Display the squad data
print(squad_data)

In [ ]:
lineups_url = "https://api-football-v1.p.rapidapi.com/v3/fixtures/lineups"
team_id = "33"
lineups_parameters = {"team": team_id}
lineups_response = requests.get(lineups_url, headers=headers, params=lineups_parameters)
lineups_data = lineups_response.json()

# Initialize a dictionary to count appearances for each player
player_appearances = {}

# Iterate through each match in the season
for match in lineups_data['response']:
    # Iterate through the starting 11 players in the home team
    for player in match['lineups']['home']['startXI']:
        player_id = player['player_id']
        if player_id in player_appearances:
            player_appearances[player_id] += 1
        else:
            player_appearances[player_id] = 1
    
    # Iterate through the starting 11 players in the away team
    for player in match['lineups']['away']['startXI']:
        player_id = player['player_id']
        if player_id in player_appearances:
            player_appearances[player_id] += 1
        else:
            player_appearances[player_id] = 1

# Sort the players by number of appearances in descending order
sorted_players = sorted(player_appearances.items(), key=lambda x: x[1], reverse=True)

# Get the top 11 players with the most appearances
top_11_players = sorted_players[:11]

# Display the top 11 players and their number of appearances
for player_id, appearances in top_11_players:
    print(f"Player ID: {player_id}, Appearances: {appearances}")

In [ ]:
top_11_players

In [ ]:
players_name=[]
years=[]
teams_id=[]
age=[]
start_year=2022
end_year=2020
teams_id = [33]
for year in range(start_year, end_year, -1):
    for team in teams_id:
        querystring = {"team":team,"season":year}
        players_response = requests.get(players_url, headers=headers, params=querystring)
        players_data=players_response.json()
        if players_data["response"]:
            team_players_data=players_data["response"]
            for player in team_players_data:
                player_name= player["player"]["name"]
                player_age=player["player"]["age"]
                players_name.append(player_name)
                years.append(year)
                teams_id.append(team)
                age.append(player_age)
data = {"Player Name": players_name,"Year": years,"Team ID": teams_id, "Age": age}
player_data_df = pd.DataFrame(data)
player_data_df

In [ ]:
for year in range(start_year, end_year - 1, -1):
    for team in teams_id:
        # Define query parameters
        querystring = {"team": team, "season": year}
        
        # Make the API request
        players_response = requests.get(players_url, headers=headers, params=querystring)
        players_data = players_response.json()
        
        # Check if response contains data
        if players_data["response"]:
            # Extract players' data for the team and season
            team_players_data = players_data["response"]
            
            # Append players' data to the list
            teams_players_data.extend(team_players_data)

# Create DataFrame from the accumulated players' data
teams_players_data_df = pd.DataFrame(teams_players_data)
teams_players_data_df

In [ ]:
players_data["response"][1]["player"]["name"]

In [ ]:
 # Check if the response contains data
            if squad_data["response"]:
                team_squad_data = squad_data["response"][0]["players"]
                for player in team_squad_data:
                    player['Team ID'] = team_id 
            all_teams_squad_data.extend(team_squad_data)
    all_teams_squad_data_df=pd.DataFrame(all_teams_squad_data)
final_pd=pd.merge(teams_df, all_teams_squad_data_df, on="Team ID", how= "outer")
final_pd